# Expressing Stochastic Optimization Problems with BARK

_Notebook by Sebastian Benthall_

This notebook is for working out the mathematical notation and API for expressing stochastic optimization problems in BARK.

Solving stochastic optimization problems will be done in a separate notebook.

As of 2023/05/03, this notebook does not have runnning Python code, as it is a revision of the earlier 'stage' based system in this repository.

## The basics of (Dynamic) Stochastic Optimization Problems

### Optimization

In an _optimization problem_, one seeks to maximize (or minimize) an objective function with respect to some control variable $x$. We can call the maximum attainable objective the _value_ $v$ of the problem.

$$v = \max_{x} r(x)$$

There are many kinds of optimization problems and solution algorithms for solving them. Common ones include linear and concave programming.

(Stachurski, 2009, and Ljungqvist and Sargent, 2018, both use $r$ for the reward function in the abstract.)

### Stochasticity

An optimization is stochastic if the value of the objective function depends on the realization of a random variable. If $m \sim P_M$ is a random variable over domain $M$, then we can optimize over the expected value of the objective function.

$$v = \max_{x} \mathbb{E}[r(x, m]$$

### Dynamism

In a dynamic optimization problem, the optimization occurs over many steps of time. Some state $s$ is realized at each time step.

$$V = \max_x \sum_{t = 0}^{T} \beta^t r(s_t, x_t)$$

Typically there is a relationship between the endogenous state values, such as $s_{t+1} = g(s_t, x_t)$. This sets up the definition of the time-dependent value function Bellman form:

$$v_t(s_t) = \max_{x_t} r(s_t, x_t) + \beta v_{t+1}(g(s_t, x_t))$$

This implies an optimal policy or decision rule:

$$\pi^*_t(s_t) = \arg \max_{x_t} r(s_t, x_t) + \beta v_{t+1}(g(s_t, x_t))$$

When $T = \infty$, the problem is recursive and it is a "dynamic programming" problem. It is then subject to many interesting theorems about its optimality conditions, which support solution techniques that exploit that the Bellman operator is a contraction.

When $T$ is finite, then the problem is not, technically speaking, a dynamic programming problem. However, under some general conditions it can be solved straightforwardly using backwards induction, i.e., solving for the value function of the last period, and then working backwards to $t = 0$. Some conditions of optimality in the dynamic programming case do not apply in the finite problem case.

We are not focused on solution methods in this notebook.


### Dynamic Stochastic Optimization Problems (DSOPs)

Straightforwardly combining the above concepts,

$$v_t(s_t) = \max_{x_t} r(m_t, s_t, x_t) + \beta \mathbb{E} [v_{t+1}(g(m_t, s_t, x_t))]$$

There is some variation in the abstract representation of DSOPs but for our purposes we will assume that $m_t$ and $m_{t+1}$ are always independent. An exogenous Markov process can be modeled with a combination of exogenous information $m_t$ and endogenous state.

## Beyond the basics

BARK is motivated by some use cases that go beyond the basics of dynamic stochastic optimization.

### Lifecycle problems

BARK is designed to support _structural changes over the agent lifecycle_. This means that the transition function $g_t$ and shocks $m_t$ can have structure that is not repeated over every period. This allows for much more expressivity in the finite problem case. This range of problems is sometimes referred to as _lifecycle problems_.

**Example.** An agent faces structurally different choices before retirement. **TODO: Equations.**

This is accomplished by decoupling the intra-period optimization problems from each other through the introduction of an end-of-period state $a_t$. Now, in the deterministic case $g: S \times X \rightarrow A$ :

$$v_t(s_t) = \max_{x_t} r(s_t, x_t) + \beta v^{+}_{t}(g(s_t, x_t))$$

Where $v^{+}_t$ is a value function over end-of-period states $a_t = g(s_t, x_t)$.

Linking the periods together is done simply: $s_{t+1} = a_t$ and $v^{+}_t(a_t) = v^{+}_{t+1}(a_{t+1})$.

If an agent experiences structural changes over time in an infinite horizon model, then these structural changes can be represented as sequential decomposition (see below). Typically, _lifecycle_ problems are finite.

### Sequential decomposition

BARK is also designed to support the _sequential decomposition_ of a time period's optimization problem into a series of simpler problems. Sequential decomposition has been shown to improve the performance of solution techniques by Lujan (2023). Here, the states, controls, and shocks for any period may be vector valued, and the problem for any one period can be defined in terms of partitions of these spaces.

**Example.** An agent makes both a consumption and a portfolio allocation choice in the same period. These choices can be separated and made in sequence. **TODO: Equations.**

**TODO: Some math here about partitioning and decomposition. To what extent can I build directly on Lujan (2023)?**

**TODO: How does decomposition of period discount factors work? Especially with stochastic discounting.**

### Dynamic discounting ('dyscounting')

**SB: I'm open to new names of this. Generalized discounting?  Sargent and Stachurski (2023) discuss stochastic discounting, time-varying discounting, state-dependent discounting. I wouldn't try to publish 'dyscounting'.** 

BARK is also designed to support _dynamic discounting_. While typically dynamic programming problems use a constant discount factor, $\beta$, there are cases where it is better to allow the generalized discount factor $\beta_t(m_t, s_t, x_t)$ to be a function of the states. (Turning $\beta$ into a function of state and controls is done by Sargent and Stachurski, 2023.)

**Example.** In a normalized version of the household consumption problem, future value is discounted as a function of the permanent income shock. **TODO: Equations.**

**Example.** A problem where probability of survival depends on allocation of resources towards health. **TODO: Equations.**

## New constructs for expressing DSOPs

The "B" in "BARK" might stand for "block". The key architectural principle of BARK is that DSOPs can be composed out of blocks, where each block contains the information needed to solve a part of the larger problem.

Currently there are just a few kinds of blocks in the BARK spec.

The idea behind 'blocks' is that new kinds of blocks can be added for other problems. For example, an M-block could handle market aggregation.

### B-blocks

A B-block is a representation of a static component of a dynamic stochastic optimization problem. They can include any of the following:

* *Optimization*. The choice of an optimal value of a control variable to optimize an objective function, subject to constraints.
* *Stochasticity*. An exogenous shock over which expectations must be taken ('prospecting').
* *Dyscounting*. Discounting of future value based on current state, choices, and shocks.


This sets up the optimization problem, given end-of-block value function $\underline{v}$, which is outside of the block and the continuation value:

$$v(s) = \max_{x \in \Gamma(s, m)} \mathbb{E}_M [r(s, m, x) + \beta(s, m, x) \underline{v}(g(s, m, x))] $$

The forward simulation looks like:
 - Start in state $s$
 - Sample $m \sim P_M$
 - Use decision rule $x = \pi(s, m)$
 - Compute reward $r(s, m, x)$
 - Transition to post-state $a = g(s, m, x)$

This can be solved in a variety of ways, depending on the conditions met by the specifics of the problem. These solution techniques will be discussed in a different notebook.

#### Formal definition

Formally, each B-block $B$ is a tuple $(\vec{S}, P_\vec{M}, \vec{X}, \Gamma, F, \vec{Y}, T, \beta)$, the agent:
 - begins in some beginning endogenous states $\vec{s} \in \vec{S}$
 - experiences some exogeneous shocks $\vec{m} \in \vec{M}$ according to probability distribution $P_\vec{M}$
     - Default: null $\emptyset$
 - can choose some actions $\vec{x} \in \vec{X}$
     - Default: null $\emptyset$
 - subject to constraints $\Gamma: \vec{S} \times \vec{M} \rightarrow \mathcal{P}(\vec{X})$ (power set over the actions)
     - For scalar actions, these may be expressed as upper and lower bounds, such that $\Gamma_{lb} \leq a \leq \Gamma_{ub}$:
         - $\Gamma_{ub}: \vec{S} \times \vec{M} \rightarrow \mathbb{R}$
         - $\Gamma_{lb}: \vec{S} \times \vec{M} \rightarrow \mathbb{R}$
         - such that $\Gamma(\vec{s}, \vec{m}) = [\Gamma_{lb}(\vec{x}, \vec{k}), \Gamma_{ub}(\vec{x}, \vec{k})]$
 - experience a reward $r: \vec{S} \times \vec{M} \times \vec{X} \rightarrow \mathbb{R}$
 - together, these determine some post-states $\vec{a} \in \vec{A}$ via...
 - a deterministic transition function $g: \vec{S} \times \vec{M} \times \vec{X} \rightarrow \vec{A}$
   - This is deterministic because shocks have been isolated to the beginning of the stage.
 - The agent has a discount factor $\beta$ for future utility.
     - This is often a constant, such as $\beta$.
     - but it can also be a function $\beta: \vec{S} \times \vec{M} \times \vec{X} \rightarrow \mathbb{R}$
     - Default: $1$.

CDC: What about a broader final transition $h(s, m, x, \underline{v})$

#### Schematized solution

The definition of the B-Block suggests that most solutions for the represented sub-problem will have a similar structure. When the corresponding model element is the default value, an operation is trivial and can be accomplished with no computation. 

You start solving a block with $v^{+}$.


| 'Move'  | Model element | Content  | Math |
|---|---|---|---|
| Expectation / Prospectation | Shock space $M$   | Computing expected values over shocks | $$v(s) = \mathbb{E}_M [v^{\sim}(s, m)]$$ |
| Optimization  | Control space $X$ | Optimize control variable with respect to future value | $$v^{\sim}(s, m) = \max_x r(s, m, x) + v^{\times}(s, m, x)$$ |
| Dyscounting  | Discount function $\beta$  | Reducing future value based on present conditions | $$v^{\times}(s, m, x) = \beta(s, m, x) \underline{v}(g(s, m, x))$$ |


**TODO: It would be better if the annotations on the value function matched the operations where it was used? But is $v^{\times}$ intuitive for end-of-period value function?$**

CDC points:
 - ~~Discounting happens between periods. Discounting should not normally happen on a stage or block.~~
 - ~~Question: is $v^{+}$ internal to the block, or outside of it?~~

#### Implementation

With a clear mathematical formalism, the information can be provided to a Python object with a simple interface like this:

```
consumption_block = BBlock(
    transition = lambda x, k, a : {'a' : x['m'] - a['c']},
    reward = lambda x, k, a : CRRA_utility(a['c'], CRRA),
    inputs = ['m'], 
    actions = ['c'],
    outputs = ['a'],
    action_upper_bound = lambda x, k: (x['m'],) , 
    action_lower_bound = lambda x, k: (0.0,) , 
    discount = .96
)
```

Or it can be parsed from a serialized configuration file like a DARK YAML.

**TODO: More expressivitiy about the input, actions, and output _spaces_. See OSE implementation.**

### P-blocks

Roughly speaking, P-blocks are sequences of one or more B-blocks, ending with a `tick` (see below).

We also introduce a simple operation, `twist`, to make it easier to reuse B-blocks.

#### Ticks

All P-blocks end with a a 'tick'.

There is some ambiguity about the meaning of the the time indicator $t$ in DSOPS when we consider the full range of problems, including dynamic programming problems, finite lifecycle problems, and sequential decomposition. There may be more than one way to decompose a problem into subproblems, and where to put the delimiting increment of $t$ can be a modeler's choice.

BARK addresses this by insisting that a P-block ends with a 'tick', which increments the model's time parameter $t$. This terminology is borrowed from NetLogo, in which `tick` measures discrete time in the global model. We will do more with `tick` as BARK expands into multi-agent systems with aggregation. At this point, it is mainly for accounting and consistency with traditional Economics modeling expecations.

- Should ticks have discounting? Conflicting intuitions: (a) simplest possible tick; (b) wanting to discount
- Does the 'tick' block manage information flow between periods and if so how?

#### Twists

We would like to support the user's reuse of B-block definitions. For example, a B-block can be defined in software code, or with a configuration file, and composed with other B-blocks developed by other users. This should facilitate reproducible research and collaboration in Economics, which is the overall goal.

Because there is no guarantee that two B-blocks $B_1$ and $B_2$ will use the same naming conventions, we allow for a simple `twist` operation, $s_{t+1} = w(a_t)$, which can be introduced as connective tissue between two B-blocks.

### As a formal grammar

The construction of DSOPs out of blocks can be thought up as a simple formal grammar a la Chomsky (1957).

Consider the following:
- Nonterminal symbols:
  - $\Omega$ - a partial DSOP with no starting condition.
  - $P_t$ - a P-block including the final tick
  - $P$ - a P-block stripped of the final tick
  - $B_w$ - a B-block that may or may not include a twist
- Terminal symbols:
  - $B$ - A B-block
  - $w$ - a 'twist' allowing for the realigning/renaming of variables $a_t = s_{t+1}$
  - $\tau$ - a 'tick' denoting the end of period and the incrementing of the $t$ counter.
  - $\alpha$ - a symbol denoting that the overall problem is repeating/recursive.
- Production rules:
  - $\Omega_\alpha \rightarrow \alpha \Omega | \Omega$, Either the problem is recursive, or it is not 
  - $\Omega \rightarrow P_t\Omega$ - you can always add a period to a 
  - $P_t \rightarrow P\tau$, ticks are mandatory to end periods
  - $P \rightarrow B_w | B_wP$
  - $B_w \rightarrow Bw | B$ twists are optional
- $\Omega_\alpha$ - a DSOP ('starting symbol'), may or may not be recursive.

This grammar can be expanded in future versions of BARK.

## References


Ljungqvist, L. and Sargent, T.J., 2018. Recursive macroeconomic theory. MIT press.

Lujan, A. 2023. _Dissertation Title_.

Sargent and Stachurski, 2023. Dynamic Programming, Volume 1: Foundations.

Stachurski, J., 2009. Economic dynamics: theory and computation. MIT Press.

Stokey, Nancy, Robert Lucas, and Prescott, Edward C. (1989) Recursive Methods in Economic Dynamics.